In [1]:
import pandas as pd
import numpy as np

### Game

In [2]:
df_game = pd.read_csv("games.csv")

In [3]:
df_game['away_team'] = df_game['away_team'].str.upper()

In [4]:
#df_game['date'] = pd.to_datetime(df_game['date'],format = '%m/%d/%Y')

In [5]:
#df_game = df_game[(df_game['date'].dt.year == 2015)]
#df_game.to_csv('filt_game.csv', index = False)

In [6]:
df_game.head()

,away_final_score,away_team,date,elapsed_time,g_id,home_final_score,delay
0,3,SLN,4/5/2015,184,201500001,0,0
1,1,ANA,4/6/2015,153,201500002,4,0
2,2,ATL,4/6/2015,156,201500003,1,16
3,6,BAL,4/6/2015,181,201500004,2,0
4,8,BOS,4/6/2015,181,201500005,0,0


In [7]:
df_game.to_csv("gameCleaned.csv", index = False)

### Pitching filter data

In [8]:
df_pitch = pd.read_csv("pitches.csv")
df_pitch.columns

Index(['type', 'pitch_type', 'event_num', 'b_score', 'ab_id', 'b_count',
       's_count', 'outs', 'pitch_num', 'on_1b', 'on_2b', 'on_3b'],
      dtype='object')

In [9]:
df_pitch.head()

,type,pitch_type,event_num,b_score,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b
0,S,FF,3,0,2015000001,0,0,0,1,0,0,0
1,S,FF,4,0,2015000001,0,1,0,2,0,0,0
2,S,FF,5,0,2015000001,0,2,0,3,0,0,0
3,B,FF,6,0,2015000001,0,2,0,4,0,0,0
4,B,CU,7,0,2015000001,1,2,0,5,0,0,0


In [10]:
#checking missing values
df_pitch.isna().sum()

type             0
pitch_type    1556
event_num        0
b_score          0
ab_id            0
b_count          0
s_count          0
outs             0
pitch_num        0
on_1b            0
on_2b            0
on_3b            0
dtype: int64

In [11]:
#dropping missing values
#df_pitch = df_pitch.dropna()

In [12]:
#dropping the small value pitches because the occurence 
#is very low which would not affect the data 
df_pitch = df_pitch[df_pitch.pitch_type != 'AB']
df_pitch = df_pitch[df_pitch.pitch_type != 'UN']
df_pitch = df_pitch[df_pitch.pitch_type != 'EP']
df_pitch = df_pitch[df_pitch.pitch_type != 'SC']
#combining the FO and PO columns because the kaggle labeled 
#FO and PO as the same 
df_pitch.pitch_type.replace('FO', 'PO', inplace=True)

##### label encode the data and convert the data to binary values to see if the pitch is a fast ball or not a fast ball

In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [14]:
df_pitch.pitch_type = df_pitch.pitch_type.astype(str)

In [15]:
df_pitch['pitch_type'] = le.fit_transform(df_pitch['pitch_type'])

In [16]:
#binarize the data to seperate the fast ball and non fastball
df_pitch['pitch_type'] =(df_pitch['pitch_type'] == 4).astype(int)

In [17]:
df_pitch = df_pitch.drop(columns = ['on_1b', 'on_2b', 'on_3b'])

In [18]:
df_pitch.to_csv('pitchingClean.csv', index = False)

### player name

In [19]:
pnf = pd.read_csv('player_names.csv')
#create a nameID to join
pnf['nameID'] = pnf['first_name'] + pnf['last_name']
#save to csv
pnf.to_csv('playerID.csv', index = False)

In [20]:
df_pname = pd.read_csv('playerID.csv')
df_pname.head()

,id,first_name,last_name,nameID
0,452657,Jon,Lester,JonLester
1,425794,Adam,Wainwright,AdamWainwright
2,457435,Phil,Coke,PhilCoke
3,435400,Jason,Motte,JasonMotte
4,519166,Neil,Ramirez,NeilRamirez


### At bat

In [21]:
df_bat = pd.read_csv('atbats.csv')
df_bat.head()

,ab_id,batter_id,event,g_id,inning,o,p_score,p_throws,pitcher_id,stand,top
0,2015000001,572761,Groundout,201500001,1,1,0,L,452657,L,True
1,2015000002,518792,Double,201500001,1,1,0,L,452657,L,True
2,2015000003,407812,Single,201500001,1,1,0,L,452657,R,True
3,2015000004,425509,Strikeout,201500001,1,2,0,L,452657,R,True
4,2015000005,571431,Strikeout,201500001,1,3,0,L,452657,L,True


In [22]:
df_b15 = pd.read_csv('batting2015.csv')

### Merging the data

##### Merge ab_id first

In [23]:
df_pc = pd.read_csv("pitchingClean.csv")
df_pc.head()

,type,pitch_type,event_num,b_score,ab_id,b_count,s_count,outs,pitch_num
0,S,1,3,0,2015000001,0,0,0,1
1,S,1,4,0,2015000001,0,1,0,2
2,S,1,5,0,2015000001,0,2,0,3
3,B,1,6,0,2015000001,0,2,0,4
4,B,0,7,0,2015000001,1,2,0,5


In [24]:
df_bat.head()

,ab_id,batter_id,event,g_id,inning,o,p_score,p_throws,pitcher_id,stand,top
0,2015000001,572761,Groundout,201500001,1,1,0,L,452657,L,True
1,2015000002,518792,Double,201500001,1,1,0,L,452657,L,True
2,2015000003,407812,Single,201500001,1,1,0,L,452657,R,True
3,2015000004,425509,Strikeout,201500001,1,2,0,L,452657,R,True
4,2015000005,571431,Strikeout,201500001,1,3,0,L,452657,L,True


In [45]:
#merge the data
df_abid = pd.merge(df_bat, df_pc, left_on = 'ab_id',right_on = 'ab_id')

##### Merge g_id from df_batpc

In [46]:
df_gid = pd.merge(df_abid, df_game, left_on = 'g_id', right_on = 'g_id')

In [47]:
df_gid.columns

Index(['ab_id', 'batter_id', 'event', 'g_id', 'inning', 'o', 'p_score',
       'p_throws', 'pitcher_id', 'stand', 'top', 'type', 'pitch_type',
       'event_num', 'b_score', 'b_count', 's_count', 'outs', 'pitch_num',
       'away_final_score', 'away_team', 'date', 'elapsed_time',
       'home_final_score', 'delay'],
      dtype='object')

In [49]:
df_gid = df_gid.drop(['o','delay','elapsed_time',
                      'stand','top', 'type',
                      'away_final_score','event_num',
                      'event', 'outs','p_score','p_throws',
                      'home_final_score','b_score',
                      'b_count','s_count'], axis = 1)

In [66]:
df_gid.to_csv("pitchCleaned.csv", index = False)

##### Merge nameID

In [59]:
df_nameID = pd.read_csv('bat.csv')
#df_nameID.head()

,playerID,yearID,PA,OBP,SLUG,BA,PCA,nameFirst,nameLast,nameGiven,debut,finalGame,bbrefID,nameID
0,abreujo02,2015.0,668.0,0.347305,0.502447,0.290375,-0.136432,Jose,Abreu,Jose Dariel,2014-03-31,2015-10-03,abreujo02,JoseAbreu
1,ackledu01,2015.0,130.5,0.301471,0.509719,0.251758,-0.109280,Dustin,Ackley,Dustin Michael,2011-06-17,2015-10-04,ackledu01,DustinAckley
2,adamecr01,2015.0,57.0,0.298246,0.301887,0.245283,0.067423,Cristhian,Adames,Cristhian Pascual,2014-07-29,2015-10-04,adamecr01,CristhianAdames
3,adamsma01,2015.0,186.0,0.279570,0.377143,0.240000,0.014556,Matt,Adams,Matthew James,2012-05-20,2015-10-04,adamsma01,MattAdams
4,adriaeh01,2015.0,132.0,0.303030,0.265487,0.185841,0.117006,Ehire,Adrianza,Ehire Enrique,2013-09-08,2015-09-22,adriaeh01,EhireAdrianza


In [62]:
df_mname = pd.merge(df_pname,df_nameID, left_on = 'nameID', right_on = 'nameID')

In [64]:
df_mname.to_csv("Mergingbatter.csv", index = False)

##### Merge Pitching to batter to complete the data

In [69]:
df_pc = pd.read_csv("pitchCleaned.csv")
df_mb = pd.read_csv("Mergingbatter.csv")

In [70]:
df_pc.head()

,ab_id,batter_id,g_id,inning,pitcher_id,pitch_type,pitch_num,away_team,date
0,2015000001,572761,201500001,1,452657,1,1,SLN,4/5/2015
1,2015000001,572761,201500001,1,452657,1,2,SLN,4/5/2015
2,2015000001,572761,201500001,1,452657,1,3,SLN,4/5/2015
3,2015000001,572761,201500001,1,452657,1,4,SLN,4/5/2015
4,2015000001,572761,201500001,1,452657,0,5,SLN,4/5/2015


In [75]:
#drop the data that are not 2015
df_pc['date'] = pd.to_datetime(df_pc['date'],format = '%m/%d/%Y')
df_pc = df_pc[(df_pc['date'].dt.year == 2015)]

In [78]:
#convert to csv file
df_pc.to_csv("pc2015.csv", index = False)

In [79]:
df_pc15 = pd.read_csv("pc2015.csv")

In [80]:
df_pc15.head()

,ab_id,batter_id,g_id,inning,pitcher_id,pitch_type,pitch_num,away_team,date
0,2015000001,572761,201500001,1,452657,1,1,SLN,2015-04-05
1,2015000001,572761,201500001,1,452657,1,2,SLN,2015-04-05
2,2015000001,572761,201500001,1,452657,1,3,SLN,2015-04-05
3,2015000001,572761,201500001,1,452657,1,4,SLN,2015-04-05
4,2015000001,572761,201500001,1,452657,0,5,SLN,2015-04-05


In [71]:
df_mb.head()

,id,first_name,last_name,nameID,playerID,yearID,PA,OBP,SLUG,BA,PCA,nameFirst,nameLast,nameGiven,debut,finalGame,bbrefID
0,452657,Jon,Lester,JonLester,lestejo01,2015.0,65.0,0.107692,0.064516,0.064516,0.410326,Jon,Lester,Jonathan Tyler,2006-06-10,2015-09-30,lestejo01
1,593372,Carlos,Martinez,CarlosMartinez,martica04,2015.0,58.0,0.172414,0.178571,0.142857,0.259954,Carlos,Martinez,Carlos Ernesto,2013-05-03,2015-09-25,martica04
2,527054,Julio,Teheran,JulioTeheran,teherju01,2015.0,54.0,0.129630,0.096154,0.096154,0.363381,Julio,Teheran,Julio Alberto,2011-05-07,2015-10-02,teherju01
3,453286,Max,Scherzer,MaxScherzer,scherma01,2015.0,71.0,0.239437,0.217391,0.217391,0.172608,Max,Scherzer,Maxwell M.,2008-04-29,2015-10-03,scherma01
4,112526,Bartolo,Colon,BartoloColon,colonba01,2015.0,60.0,0.150000,0.155172,0.137931,0.290670,Bartolo,Colon,Bartolo,1997-04-04,2015-10-04,colonba01


In [90]:
df_merge = pd.merge(df_pc15, df_mb, left_on = 'batter_id', right_on = 'id')

In [91]:
df_merge = df_merge.drop(columns = ['nameGiven','first_name','last_name','bbrefID','pitch_num'])

In [92]:
#save as csv 
df_merge.to_csv("pitchbatcomplete.csv", index = False)

In [93]:
df_merge.columns

Index(['ab_id', 'batter_id', 'g_id', 'inning', 'pitcher_id', 'pitch_type',
       'away_team', 'date', 'id', 'nameID', 'playerID', 'yearID', 'PA', 'OBP',
       'SLUG', 'BA', 'PCA', 'nameFirst', 'nameLast', 'debut', 'finalGame'],
      dtype='object')

In [94]:
df = pd.read_csv('pitchbatcomplete.csv')

In [95]:
df.head()

,ab_id,batter_id,g_id,inning,pitcher_id,pitch_type,away_team,date,id,nameID,...,yearID,PA,OBP,SLUG,BA,PCA,nameFirst,nameLast,debut,finalGame
0,2015000001,572761,201500001,1,452657,1,SLN,2015-04-05,572761,MattCarpenter,...,2015.0,665.0,0.365414,0.505226,0.271777,-0.139731,Matt,Carpenter,2011-06-04,2015-10-04
1,2015000001,572761,201500001,1,452657,1,SLN,2015-04-05,572761,MattCarpenter,...,2015.0,665.0,0.365414,0.505226,0.271777,-0.139731,Matt,Carpenter,2011-06-04,2015-10-04
2,2015000001,572761,201500001,1,452657,1,SLN,2015-04-05,572761,MattCarpenter,...,2015.0,665.0,0.365414,0.505226,0.271777,-0.139731,Matt,Carpenter,2011-06-04,2015-10-04
3,2015000001,572761,201500001,1,452657,1,SLN,2015-04-05,572761,MattCarpenter,...,2015.0,665.0,0.365414,0.505226,0.271777,-0.139731,Matt,Carpenter,2011-06-04,2015-10-04
4,2015000001,572761,201500001,1,452657,0,SLN,2015-04-05,572761,MattCarpenter,...,2015.0,665.0,0.365414,0.505226,0.271777,-0.139731,Matt,Carpenter,2011-06-04,2015-10-04
